In [ ]:
# -------------------------------------------------------------------------------- NOTEBOOK-CELL: CODE
# IMPORT===============================================================================
# =====================================================================================
# -*- coding: utf-8 -*-
import json
import logging
import math
import os
import shutil
import tempfile
import traceback
from datetime import datetime
from pathlib import Path
from typing import Optional, Tuple, List, Dict

# Import third-party libraries
import pandas as pd
import numpy as np
from dataiku import pandasutils as pdu

# Import Dataiku-specific modules
import dataiku
from dataiku.scenario import Scenario

# -------------------------------------------------------------------------------- NOTEBOOK-CELL: CODE
# IMPORTAZIONI LIBRERIE SISTEMA=========================================================
# ======================================================================================

import os
import sys
import json
import traceback
import shutil
import time
import logging
from pathlib import Path
from datetime import datetime
import numpy as np
from typing import List, Tuple, Dict, Optional, Union
import importlib
# Librerie geospaziali - Stack GDAL/Rasterio
try:
    import rasterio
    from rasterio.transform import from_bounds, Affine
    from rasterio.warp import calculate_default_transform, reproject, Resampling
    from rasterio.merge import merge
    RASTERIO_AVAILABLE = True
except ImportError:
    RASTERIO_AVAILABLE = False
    logging.info("ERRORE: Stack rasterio/GDAL non disponibile")

# ======================================================================================
# IMPORTAZIONI MODULI SPECIALIZZATI
# ======================================================================================

try:

    from crolli_modules import mosaic_operations
    from crolli_modules import alignment_operations
    from crolli_modules import resolution_operations
    from crolli_modules import io_operations
    from crolli_modules import metadata_operations
    importlib.reload(io_operations)
    EXTERNAL_MODULES_AVAILABLE = True
    logging.info("Moduli specializzati caricati correttamente")
except ImportError as e:
    EXTERNAL_MODULES_AVAILABLE = False
    logging.error(f"ERRORE: Moduli specializzati non disponibili: {e}")
    logging.error("Moduli richiesti: mosaic_operations.py, alignment_operations.py,")
    logging.error("resolution_operations.py, io_operations.py, metadata_operations.py")

# -------------------------------------------------------------------------------- NOTEBOOK-CELL: CODE
# FUNZIONI GENERALI
client = dataiku.api_client()
project = client.get_default_project()

def _set_project_variable(key, value):
    variables = project.get_variables()
    # Sostituisci NaN (sia float che stringa) con stringa vuota
    if value is None or (isinstance(value, float) and np.isnan(value)) or str(value).lower() == "nan":
        value = ""
    try:
        variables['local'][key] = value.strip() if isinstance(value, str) else value
    except Exception:
        variables['local'][key] = value
    project.set_variables(variables)
    logging.debug(f"settata variabile {key} = {value}")

# -------------------------------------------------------------------------------- NOTEBOOK-CELL: CODE
#  CLASSE CONFIGURAZIONE SISTEMA========================================================
# ======================================================================================
class MergeDatasetConfig:
    """
    Configurazione centralizzata per merge dataset geospaziali.

    Gestisce parametri operativi, formati supportati e configurazioni
    per garantire compatibilità con standard industriali GIS.
    """

    def __init__(self, payload: dict = None):
        """Inizializza configurazione importando parametri globali."""
        for key, value in globals().items():
            if key.isupper() and not key.startswith('_'):
                setattr(self, key, value)
        self.SUPPORTED_EXTENSIONS = [
            ".tif", ".tiff",           # GeoTIFF
            ".img",                    # ERDAS Imagine
            ".bil", ".bip", ".bsq",    # ENVI band-interleaved
            ".adf",                    # Esri Grid
            ".jp2", ".j2k", ".jpx",    # JPEG 2000
            ".vrt",                    # Virtual Raster GDAL
            ".hdf", ".h5",             # Hierarchical Data Format
            ".nc",                     # NetCDF
            ".asc",                    # ASCII Grid ESRI
            ".grd", ".flt", ".rst", ".pix"  # Formati aggiuntivi
        ]

        self.MERGE_SUBFOLDER = "MERGE"
        self.TEMP_SUBFOLDER = "TEMP_MERGE"

        # Valori sensati di default
        self.INPUT_FOLDER_PRE: Optional[str] = None
        self.INPUT_FOLDER_POST: Optional[str] = None
        self.OUTPUT_FOLDER: Optional[str] = None

        # Oggetti Dataiku Folder risolti (se disponibili)
        self.INPUT_FOLDER_PRE_OBJ = None
        self.INPUT_FOLDER_POST_OBJ = None
        self.OUTPUT_FOLDER_OBJ = None

        # Percorsi locali temporanei dove abbiamo scaricato le folder (se richiesto)
        self.INPUT_FOLDER_PRE_TMP: Optional[str] = None
        self.INPUT_FOLDER_POST_TMP: Optional[str] = None
        self.OUTPUT_FOLDER_TMP: Optional[str] = None

        # Tieni traccia delle tmp dirs create per cleanup
        self._tmp_dirs: List[str] = []

        self.RESAMPLING_STRATEGY = "best"
        self.RESAMPLING_METHOD = "bilinear"
        self.MERGE_METHOD = "mean"
        self.NODATA_VALUE = -9999
        self.ALIGN_PIXELS = True
        self.CREATE_HDR_FILES = True
        self.COMPRESSION = "lzw"
        self.TILED = True
        self.BLOCK_SIZE = 256
        self.OVERWRITE_EXISTING = True
        self.ENABLE_NAME_FILTERING = False
        self.NAME_FILTER_PATTERN = ""

        # Rilevazione semplice di moduli esterni utili (rasterio/gdal)
        self.EXTERNAL_MODULES_AVAILABLE = False
        if 'detect_external_modules' in globals() and detect_external_modules:
            try:
                import rasterio  # noqa: F401
                self.EXTERNAL_MODULES_AVAILABLE = True
            except Exception:
                pass

        # Dataiku client import (lazy, may not exist in non-DSS env)
        self._dataiku_available = False
        self._dataiku = None
        try:
            import dataiku  # type: ignore
            self._dataiku_available = True
            self._dataiku = dataiku
        except Exception:
            pass
        print(payload)
        # Se è stato passato un payload, parsalo
        if payload:
            self._load_from_payload(payload)
#           
    def _load_from_payload(self, payload: dict):

        # Campi semplici a livello root: li mappiamo su attributi uppercase con nomi coerenti
        mapping = {
            "resampling_strategy": "RESAMPLING_STRATEGY",
            "resampling_method": "RESAMPLING_METHOD",
            "merge_method": "MERGE_METHOD",
            "nodata_value": "NODATA_VALUE",
            "align_pixels": "ALIGN_PIXELS",
            "create_hdr_files": "CREATE_HDR_FILES",
            "compression": "COMPRESSION",
            "tiled": "TILED",
            "block_size": "BLOCK_SIZE",
            "overwrite_existing": "OVERWRITE_EXISTING",
            "enable_name_filtering": "ENABLE_NAME_FILTERING",
            "name_filter_pattern": "NAME_FILTER_PATTERN",

            "damage_threshold": "DAMAGE_THRESHOLD",
            "min_overlap_percent": "MIN_OVERLAP_PERCENT",
            "height_field_name": "HEIGHT_FIELD_NAME",
            "collapse_threshold_percent": "COLLAPSE_THRESHOLD_PERCENT",
            "altezza_build": "ALTEZZA_BUILD",
            "type_costr": "TYPE_COSTR",
            "sup_field": "SUP_FIELD",
            "fid_field": "FID_FIELD",

            "elab_id": "ELAB_ID",
            "input_folder_pre": "INPUT_FOLDER_PRE",
            "input_folder_post": "INPUT_FOLDER_POST",
            "output_folder": "OUTPUT_FOLDER",
            "dsm_pre_path": "DSM_PRE_PATH",
            "dsm_post_path": "DSM_POST_PATH",
            "buildings_path": "BUILDINGS_PATH",
            "output_directory_bdd": "OUTPUT_DIRECTORY_BDD"
                }

        for src_key, attr in mapping.items():

            if src_key in payload:
                val = payload[src_key]
                print(attr, val)
                # Trattamenti specifici
                if attr in ("ALIGN_PIXELS", "CREATE_HDR_FILES", "TILED", "OVERWRITE_EXISTING", "ENABLE_NAME_FILTERING"):
                    setattr(self, attr, self._to_bool(val))
                elif attr == "BLOCK_SIZE":
                    setattr(self, attr, self._to_int(val, default=self.BLOCK_SIZE))
                else:
                    setattr(self, attr, val)


    def print_config(self):
        """Visualizza configurazione corrente del sistema."""
        print(f"\nCORE ENGINE PROCESSING - CONFIGURAZIONE")
        print(f"{'='*60}")
        print(f"Dataset PRE:  {(self.INPUT_FOLDER_PRE) if self.INPUT_FOLDER_PRE else 'N/A'}")
        print(f"Dataset POST: {(self.INPUT_FOLDER_POST) if self.INPUT_FOLDER_POST else 'N/A'}")
        print(f"Output: {(self.OUTPUT_FOLDER) if self.OUTPUT_FOLDER else 'N/A'}")
        print(f"")
        print(f"PARAMETRI PROCESSING:")
        print(f"  Risoluzione: {self.RESAMPLING_STRATEGY} ({self.RESAMPLING_METHOD})")
        print(f"  Sovrapposizioni: {self.MERGE_METHOD}")
        print(f"  NoData: {self.NODATA_VALUE}")
        print(f"  Allineamento: {'ON' if self.ALIGN_PIXELS else 'OFF'}")
        print(f"  Metadati HDR: {'ON' if self.CREATE_HDR_FILES else 'OFF'}")
        print(f"")
        print(f"SISTEMA:")
        print(f"  Formati supportati: {len(self.SUPPORTED_EXTENSIONS) if hasattr(self, 'SUPPORTED_EXTENSIONS') else 'N/A'} tipologie")
        print(f"  Moduli: {'OPERATIVI' if self.EXTERNAL_MODULES_AVAILABLE else 'ERRORE'}")
        print(f"  Compressione: {self.COMPRESSION.upper()}")
        print(f"  TIFF: {'TILED' if self.TILED else 'STRIP'} ({self.BLOCK_SIZE}px)")
        if self.ENABLE_NAME_FILTERING:
            print(f"  Filtro naming: {self.NAME_FILTER_PATTERN}")
        else:
            print(f"  Filtro naming: DISABILITATO (tutti i file)")

    def _to_bool(self,val):
        """Converte diverse rappresentazioni in booleano in modo sicuro."""
        if isinstance(val, bool):
            return val
        if val is None:
            return False
        if isinstance(val, (int, float)):
            return bool(val)
        s = str(val).strip().lower()
        if s in ("false", "0", "no", "off", ""):
            return False
        return True

    def _to_int(self, value, default=0):
        """Converte un valore in intero, con valore predefinito in caso di errore."""
        try:
            return int(value)
        except (ValueError, TypeError):
            return default

    def _is_missing(self, value):
        """Verifica se un valore è mancante (None o NaN)."""
        return value is None or (isinstance(value, float) and value != value)

# -------------------------------------------------------------------------------- NOTEBOOK-CELL: CODE
#  FUNZIONI COORDINAMENTO MODULI =======================================================
# ======================================================================================

def determine_utm_zone_for_italy(files_info: List[Dict]) -> str:
    """
    Determina la zona UTM ottimale per il territorio italiano analizzando i bounds dei dataset.

    Analizza le coordinate per scegliere tra:
    - EPSG:32632 (UTM 32N) per Italia Occidentale
    - EPSG:32633 (UTM 33N) per Italia Centro-Orientale

    Args:
        files_info: Lista metadati files con bounds

    Returns:
        str: Codice EPSG UTM ottimale per la zona geografica
    """
    try:
        # Raccogli coordinate geografiche dai file disponibili
        geographic_coords = []
        files_analyzed = 0
        files_with_bounds = 0
        files_with_crs = 0

        print(f"   Analisi {len(files_info)} file per determinazione UTM...")

        for file_info in files_info:
            files_analyzed += 1
            file_name = file_info.get('name', 'Unknown')

            # Verifica disponibilità bounds
            if 'bounds' not in file_info or not file_info['bounds']:
                print(f"   {file_name}: bounds mancanti")
                continue
            files_with_bounds += 1

            # Verifica disponibilità CRS
            if 'crs' not in file_info or not file_info['crs']:
                print(f"   {file_name}: CRS mancante")
                continue
            files_with_crs += 1

            bounds = file_info['bounds']
            crs = file_info['crs']

            # Calcola centro del dataset
            center_x = (bounds[0] + bounds[2]) / 2
            center_y = (bounds[1] + bounds[3]) / 2

            print(f"   {file_name}: centro ({center_x:.1f}, {center_y:.1f}), CRS: {str(crs)[:20]}...")

            # Coordinate geografiche WGS84
            if (isinstance(crs, object) and hasattr(crs, 'to_epsg') and
                crs.to_epsg() == 4326) or str(crs) == 'EPSG:4326':
                geographic_coords.append(center_x)  # longitudine
                print(f"      Coordinate geografiche: {center_x:.3f}°E, {center_y:.3f}°N")
            elif (center_x > -180 and center_x < 180 and
                  center_y > -90 and center_y < 90):
                # Sembrano coordinate geografiche
                geographic_coords.append(center_x)
                print(f"   Coordinate geografiche: {center_x:.3f}°, {center_y:.3f}°")
            else:
                # Coordinate proiettate - analisi euristica per sistemi italiani
                print(f"   Coordinate proiettate: {center_x:.0f}, {center_y:.0f}")

                # Euristica per sistemi coordinate italiani comuni:
                # - UTM 32N Italia Occidentale: X ~ 400,000-700,000
                # - UTM 33N Italia Centro-Orientale: X ~ 200,000-600,000
                # - Gauss-Boaga Ovest: X ~ 1,400,000-1,700,000
                # - Gauss-Boaga Est: X ~ 2,300,000-2,700,000

                if 1400000 <= center_x <= 1700000:
                    # Gauss-Boaga Ovest → UTM 32N
                    geographic_coords.append(9.0)  # Approssimazione Italia Occidentale
                    print(f"   Gauss-Boaga Ovest rilevato → UTM 32N suggerito")
                elif 2300000 <= center_x <= 2700000:
                    # Gauss-Boaga Est → UTM 33N
                    geographic_coords.append(15.0)  # Approssimazione Italia Centro-Orientale
                    print(f"   Gauss-Boaga Est rilevato → UTM 33N suggerito")
                elif 400000 <= center_x <= 700000:
                    # Potenziale UTM 32N
                    geographic_coords.append(9.0)
                    print(f"   Possibile UTM 32N → confermato")
                elif 200000 <= center_x <= 600000:
                    # Potenziale UTM 33N
                    geographic_coords.append(15.0)
                    print(f"   Possibile UTM 33N → confermato")
        # Report diagnostico
        print(f"   Diagnostica: {files_analyzed} file analizzati, {files_with_bounds} con bounds, {files_with_crs} con CRS")
        print(f"   Coordinate geografiche raccolte: {len(geographic_coords)}")

        if not geographic_coords:
            print("   Nessuna coordinata analizzabile, uso default UTM 32N")
            return "EPSG:32632"

        # Calcola longitudine media
        avg_longitude = sum(geographic_coords) / len(geographic_coords)
        print(f"   Longitudine media stimata: {avg_longitude:.1f}°")

        # Decisione finale zona UTM per Italia
        if avg_longitude < 12.0:
            selected_utm = "EPSG:32632"  # UTM 32N
            zone_name = "UTM 32N (Italia Occidentale)"
        else:
            selected_utm = "EPSG:32633"  # UTM 33N
            zone_name = "UTM 33N (Italia Centro-Orientale)"

        print(f"   UTM selezionato: {selected_utm} - {zone_name}")
        return selected_utm

    except Exception as e:
        print(f"   Errore determinazione UTM: {e}")
        print("   Fallback default: EPSG:32632 (UTM 32N)")
        return "EPSG:32632"

def get_crs_from_first_file(files_info: List[Dict]) -> Optional[str]:
    """
    Estrae CRS dal primo file di un dataset in modo semplice e diretto.

    Args:
        files_info: Lista metadati files da io_operations.analyze_datasets()

    Returns:
        str: CRS in formato string (es. "EPSG:25832") o None se non trovato
    """
    if not files_info:
        return None

    first_file = files_info[0]
    crs = first_file.get('crs')

    if crs is not None:
        crs_string = str(crs)
        print(f"   CRS estratto: {crs_string}")
        return crs_string
    else:
        print(f"   CRS non trovato nel file: {first_file.get('path', 'Unknown')}")
        return None

def manage_crs_simplified(pre_info: List[Dict], post_info: List[Dict]) -> str:
    """
    Gestione CRS automatica con priorità PRE → POST → fallback geografico.

    Logica di priorità:
    1. Se PRE ha CRS valido → usa quello (priorità assoluta)
    2. Se PRE non ha CRS ma POST sì → usa CRS di POST
    3. Altrimenti → determina UTM ottimale per Italia basandosi sulla geografia

    Args:
        pre_info: Metadati dataset PRE da io_operations
        post_info: Metadati dataset POST da io_operations

    Returns:
        str: CRS target per processing (sempre valido)
    """
    print("Gestione CRS con priorità PRE → POST → geografico...")

    # PRIORITÀ 1: Estrai CRS da dataset PRE
    print("Estrazione CRS dataset PRE...")
    pre_crs = get_crs_from_first_file(pre_info)

    if pre_crs:
        print(f"PRE ha CRS valido: {pre_crs} → UTILIZZATO (priorità assoluta)")
        return pre_crs

    print("PRE senza CRS, controllo POST...")

    # PRIORITÀ 2: Se PRE non ha CRS, prova POST
    print("Estrazione CRS dataset POST...")
    post_crs = get_crs_from_first_file(post_info)

    if post_crs:
        print(f"POST ha CRS valido: {post_crs} → UTILIZZATO (fallback POST)")
        return post_crs

    print("Né PRE né POST hanno CRS validi")

    # PRIORITÀ 3: Fallback intelligente basato su geografia
    print("Determinazione UTM ottimale per Italia...")

    # Usa tutti i file disponibili per determinare la zona geografica
    all_files = pre_info + post_info
    fallback_crs = determine_utm_zone_for_italy(all_files)

    print(f"Fallback geografico: {fallback_crs}")
    return fallback_crs

# -------------------------------------------------------------------------------- NOTEBOOK-CELL: CODE
# WORKFLOW PRINCIPALE SISTEMA===========================================================
# ======================================================================================
def run_workflow(payload):
    """
    Workflow principale per elaborazione dataset DSM/DTM.

    Esegue il pipeline di processing attraverso l'architettura modulare:
    1. Verifica dipendenze sistema
    2. Scansione dataset input
    3. Analisi metadati e validazione
    4. Gestione sistemi coordinate (CRS)
    5. Mosaicatura dataset
    6. Allineamento pixel
    7. Generazione metadati professionali

    Returns:
        tuple: (success: bool, output_files_count: int) - True/numero file se completato con successo, False/0 in caso di errori
    """
    start_time = time.time()

    print("\n" + "="*60)
    print("CORE ENGINE PROCESSING - AVVIO SISTEMA")
    print("="*60)

    try:
        # Verifica dipendenze
        print("\nVerifica dipendenze sistema...")

        if not RASTERIO_AVAILABLE:
            print("ERRORE: Stack rasterio/GDAL non disponibile")
            return False, 0

        if not EXTERNAL_MODULES_AVAILABLE:
            print("ERRORE: Moduli specializzati non disponibili")
            return False, 0

        print("Dipendenze verificate correttamente")

        # Inizializzazione configurazione
        print("\nInizializzazione configurazione...")
        config = MergeDatasetConfig(payload)

        config.print_config()

#         # Estrai i valori dinamicamente da config.OUTPUT_FOLDER
#         output_base, output_subfolder = os.path.split(config.OUTPUT_FOLDER)

#         # Usa una directory temporanea per il percorso base
#         if not os.path.isabs(output_base):
#             output_base = os.path.join(tempfile.gettempdir(), output_base.lstrip('/'))

#         # Aggiorna il percorso di OUTPUT_FOLDER
#         config.OUTPUT_FOLDER = os.path.join(output_base, output_subfolder)
#         Path(config.OUTPUT_FOLDER).mkdir(parents=True, exist_ok=True)

#         print(f"✅ Directory di output configurata dinamicamente: {config.OUTPUT_FOLDER}")


        # FASE 1: Scansione dataset
        print("\n" + "="*60)
        print("FASE 1/6: SCANSIONE DATASET")
        print("="*60)
        folder = dataiku.Folder("minio_input")
        file_list = folder.list_paths_in_partition()
        with tempfile.TemporaryDirectory() as tmpdir:
            pre_folder = os.path.join(tmpdir, "pre_folder")
            post_folder = os.path.join(tmpdir, "post_folder")
            temp_output_folder =os.path.join(tmpdir, "output")


            os.makedirs(pre_folder, exist_ok=True)
            os.makedirs(post_folder, exist_ok=True)

            pre_file_found = False
            post_file_found = False

            print(f"Cartella temporanea creata: {tmpdir}")

            for f in file_list:
                print(f.lower())
                print(config.INPUT_FOLDER_PRE)
                if f:
                    if f.lower() == config.INPUT_FOLDER_PRE.lower():
                        pre_file = os.path.join(pre_folder, os.path.basename(f))
                        with folder.get_download_stream(f) as stream, open(pre_file, 'wb') as out:
                            out.write(stream.read())
                        pre_file_found = True
                        print(f"✅ File scaricato in pre_folder: {pre_file}")

                    if f.lower() == config.INPUT_FOLDER_POST.lower():
                        post_file = os.path.join(post_folder, os.path.basename(f))
                        with folder.get_download_stream(f) as stream, open(post_file, 'wb') as out:
                            out.write(stream.read())
                        post_file_found = True
                        print(f"✅ File scaricato in post_folder: {post_file}")

            # Verifica il contenuto delle directory
            print("Contenuto di pre_folder:")
            print(os.listdir(pre_folder))
            print("Contenuto di post_folder:")
            print(os.listdir(post_folder))

            # Verifica se i file sono stati scaricati
            if pre_file_found and os.path.isdir(pre_folder):
                config.INPUT_FOLDER_PRE = pre_folder
                print(f"✅ Percorso valido per INPUT_FOLDER_PRE: {pre_folder}")
            else:
                print(f"❌ Percorso non trovato 111: {pre_folder}")
                raise ValueError("File per INPUT_FOLDER_PRE non trovato o directory non valida.")

            if post_file_found and os.path.isdir(post_folder):
                config.INPUT_FOLDER_POST = post_folder
                print(f"✅ Percorso valido per INPUT_FOLDER_POST: {post_folder}")
            else:
                print(f"❌ Percorso non trovato 222: {post_folder}")
                raise ValueError("File per INPUT_FOLDER_POST non trovato o directory non valida.")

            # Sposta il codice che utilizza pre_folder_or_path e post_folder_or_path qui
            pre_folder_or_path = config.INPUT_FOLDER_PRE
            post_folder_or_path = config.INPUT_FOLDER_POST

            if os.path.isdir(pre_folder_or_path):
                print("Contenuto di pre_folder_or_path:")
                for item in os.listdir(pre_folder_or_path):
                    print(f"  - {item}")
            else:
                print(f"pre_folder_or_path non è una directory: {pre_folder_or_path}")

            if os.path.isdir(post_folder_or_path):
                print("Contenuto di post_folder_or_path:")
                for item in os.listdir(post_folder_or_path):
                    print(f"  - {item}")
            else:
                print(f"post_folder_or_path non è una directory: {post_folder_or_path}")

            # ottieni i percorsi locali da passare a io_operations
            pre_folder_or_path = config.INPUT_FOLDER_PRE
            post_folder_or_path = config.INPUT_FOLDER_POST
            print(f"pre_folder_or_path:  {pre_folder_or_path}")
            print(f"post_folder_or_path:  {post_folder_or_path}")


            pre_files = io_operations.scan_dataset_files(pre_folder_or_path, config)
            post_files = io_operations.scan_dataset_files(post_folder_or_path, config)

            if not pre_files or not post_files:
                print("ERRORE: Dataset PRE o POST non contengono file validi")
                print(f"Files PRE: {len(pre_files) if pre_files else 0}")
                print(f"Files POST: {len(post_files) if post_files else 0}")
                return False, 0

            print(f"Scansione completata")
            print(f"Dataset PRE: {len(pre_files)} file")
            print(f"Dataset POST: {len(post_files)} file")

            # FASE 2: Analisi metadati
            print("\n" + "="*60)
            print("FASE 2/6: ANALISI METADATI")
            print("="*60)

            print("Analisi metadati dataset...")

            pre_info, post_info = io_operations.analyze_datasets(pre_files, post_files, config)

            if not pre_info or not post_info:
                print("ERRORE: Impossibile analizzare metadati dataset")
                return False, 0

            print("Analisi metadati completata")
            print(f"Dataset PRE: {len(pre_info)} file analizzati")
            print(f"Dataset POST: {len(post_info)} file analizzati")

            # FASE 3: Gestione CRS
            print("\n" + "="*60)
            print("FASE 3/6: GESTIONE SISTEMI COORDINATE")
            print("="*60)

            print("Gestione sistemi coordinate...")

            target_crs = manage_crs_simplified(pre_info, post_info)
            # NOTA: La nuova gestione restituisce SEMPRE un CRS valido (con fallback automatico)

            print("Gestione CRS completata")
            print(f"Sistema coordinate target: {target_crs}")

            # FASE 3.5: Determinazione risoluzione spaziale
            print("\n" + "="*60)
            print("FASE 3.5/6: DETERMINAZIONE RISOLUZIONE SPAZIALE")
            print("="*60)

            print("Analisi risoluzioni dataset...")

            # Estrai risoluzione rappresentativa per PRE e POST
            pre_resolution = resolution_operations.get_most_frequent_resolution(pre_info)
            post_resolution = resolution_operations.get_most_frequent_resolution(post_info)

            if pre_resolution is None or post_resolution is None:
                print("ERRORE: Impossibile estrarre risoluzioni dai dataset")
                return False, 0

            print(f"Risoluzione PRE dominante: {pre_resolution:.3f}m")
            print(f"Risoluzione POST dominante: {post_resolution:.3f}m")

            # Determina risoluzione finale usando resolution_operations
            target_resolution = resolution_operations.determine_final_resolution_between_datasets(
                pre_resolution, post_resolution, config
            )

            if target_resolution is None:
                print("ERRORE: Impossibile determinare risoluzione target")
                return False, 0

            print(f"Risoluzione target determinata: {target_resolution}m")
            print(f"Strategia utilizzata: {config.RESAMPLING_STRATEGY}")

            # FASE 4: Mosaicatura
            print("\n" + "="*60)
            print("FASE 4/6: MOSAICATURA DATASET")
            print("="*60)

            print("Mosaicatura dataset...")

#             Path(config.OUTPUT_FOLDER).mkdir(parents=True, exist_ok=True)
#             merge_folder = Path(config.OUTPUT_FOLDER) / config.MERGE_SUBFOLDER
            temp_output_folder = Path(temp_output_folder)
            merge_folder = temp_output_folder / config.MERGE_SUBFOLDER
#             merge_folder.mkdir(exist_ok=True)
            os.makedirs(merge_folder, exist_ok=True)


            pre_mosaic_path = str(merge_folder / "merged_PRE.tif")
            post_mosaic_path = str(merge_folder / "merged_POST.tif")

            print(f"Output: {merge_folder}")

            pre_result = mosaic_operations.create_mosaic(pre_info, pre_mosaic_path, target_crs, config, "PRE", target_resolution)
            post_result = mosaic_operations.create_mosaic(post_info, post_mosaic_path, target_crs, config, "POST", target_resolution)

            if not pre_result or not post_result:
                print("ERRORE: Fallimento mosaicatura")
                return False, 0

            print("Mosaicatura completata")
            print(f"Dataset PRE: {Path(pre_result).name}")
            print(f"Dataset POST: {Path(post_result).name}")

            # FASE 5: Allineamento
            print("\n" + "="*60)
            print("FASE 5/6: ALLINEAMENTO GRIGLIA PIXEL")
            print("="*60)

            if config.ALIGN_PIXELS:
                print("Allineamento griglia pixel...")

                try:
                    aligned_pre, aligned_post = alignment_operations.align_grids(pre_result, post_result, config)
                    if aligned_pre and aligned_post:
                        pre_result, post_result = aligned_pre, aligned_post
                        print("Allineamento completato")
                    else:
                        print("Allineamento non riuscito, utilizzo mosaici originali")
                except ValueError as e:
                    if "Nessuna sovrapposizione geometrica" in str(e):
                        print("\n🚨 PROCESSO INTERROTTO: NESSUNA SOVRAPPOSIZIONE GEOMETRICA")
                        print("Il workflow è stato terminato in modo controllato.")
                        print("Consultare i dettagli sopra per la risoluzione del problema.")
                        return False, 0
                    else:
                        # Altro tipo di ValueError, ri-solleva
                        raise
            else:
                print("Allineamento disattivato")

            # FASE 6: Metadati
            print("\n" + "="*60)
            print("FASE 6/6: GENERAZIONE METADATI")
            print("="*60)

            if config.CREATE_HDR_FILES:
                print("Generazione metadati HDR...")

                metadata_operations.create_hdr_files(pre_result, post_result, config)

                print("Metadati HDR generati")
            else:
                print("Generazione metadati disattivata")

            # Report finale
            processing_time = time.time() - start_time

            # Conta i file di output generati
            output_files = []
#             output_folder = Path(config.OUTPUT_FOLDER)

            # File principali (.tif)
            if os.path.exists(pre_result):
                output_files.append(pre_result)
            if os.path.exists(post_result):
                output_files.append(post_result)

            # File metadati (.hdr)
            if config.CREATE_HDR_FILES:
                pre_hdr = pre_result.replace('.tif', '.hdr')
                post_hdr = post_result.replace('.tif', '.hdr')
                if os.path.exists(pre_hdr):
                    output_files.append(pre_hdr)
                if os.path.exists(post_hdr):
                    output_files.append(post_hdr)

            print("\n" + "="*60)
            print("ELABORAZIONE COMPLETATA")
            print("="*60)
            print(f"Dataset PRE:  {Path(pre_result).name}")
            print(f"Dataset POST: {Path(post_result).name}")
            print(f"Tempo elaborazione: {processing_time:.1f} secondi")
            print(f"Sistema: Core Engine Processing")

            # Esporta i file dalla cartella temporanea alla cartella di output con Dataiku
            output_folder = dataiku.Folder("output_preprocessing_raster")
            output_folder_path = output_folder.get_path()

            # Determina codice di partition (es. "202509231300") a partire dal percorso originale
            base_code = None
            base_code = payload.get('elab_id')

            # Create target partitioned MERGE folder inside managed folder
            partition_merge_dir = os.path.join(output_folder_path, base_code, config.MERGE_SUBFOLDER)
            os.makedirs(partition_merge_dir, exist_ok=True)

            # Copia file e directory dal temp_output_folder nella cartella partitioned/MERGE
            for item_name in os.listdir(temp_output_folder):
                src_path = os.path.join(temp_output_folder, item_name)
                dest_path = os.path.join(partition_merge_dir, item_name)

                if os.path.isfile(src_path):
                    shutil.copy2(src_path, dest_path)
                elif os.path.isdir(src_path):
                    shutil.copytree(src_path, dest_path, dirs_exist_ok=True)

            print(f"✅ Dati esportati (file e directory) nella cartella di output partitioned: {partition_merge_dir}")

            # --- NUOVA LOGICA: impostazione variabili locali per la recipe successiva ---
            try:
                # Costruisci link relativi usati dalla recipe successiva (partition/MERGE/filename)
                pre_filename = os.path.basename(pre_result) if pre_result else None
                post_filename = os.path.basename(post_result) if post_result else None

                if pre_filename:
                    dsm_pre_link = os.path.join(base_code, config.MERGE_SUBFOLDER, pre_filename)
                    # _set_project_variable("DSM_PRE_PATH", dsm_pre_link)
                    config.DSM_PRE_PATH = dsm_pre_link
                else:
                    dsm_pre_link = None

                if post_filename:
                    dsm_post_link = os.path.join(base_code, config.MERGE_SUBFOLDER, post_filename)
                    # _set_project_variable("DSM_POST_PATH", dsm_post_link)
                    config.DSM_POST_PATH = dsm_post_link
                else:
                    dsm_post_link = None

                print(f"✅ Variabili locali impostate:")
                if dsm_pre_link:
                    print(f"   DSM_PRE_PATH = {dsm_pre_link}")
                if dsm_post_link:
                    print(f"   DSM_POST_PATH = {dsm_post_link}")

            except Exception as ex:
                print(f"⚠️ Errore impostazione variabili locali: {ex}")
            # --- fine nuova logica ---


            return True, len(output_files), config

    except Exception as e:
        print(f"\nERRORE SISTEMA: {str(e)}")
        traceback.print_exc()
        return False, 0

# -------------------------------------------------------------------------------- NOTEBOOK-CELL: CODE
def _create_payload_from_config_tables():
    payload = {}

    # --- leggi tabella 'configurazione'
    try:
        conf_df = dataiku.Dataset('configurazione').get_dataframe()
        for _, row in conf_df.iterrows():
            var_name = row.get("variabile")
            var_value = row.get("valore")

            # prova a interpretare JSON nel valore
            if isinstance(var_value, str):
                try:
                    var_value = json.loads(var_value)
                except Exception:
                    pass

            if var_name:  # aggiungi solo se esiste il nome
                payload[var_name] = var_value
    except Exception as ex:
        logging.warning(f"Errore lettura tabella 'configurazione': {ex}")

    logging.debug("Creazione del payload completata")
    return payload

# -------------------------------------------------------------------------------- NOTEBOOK-CELL: CODE
def _prepare_scenario_payload(payload: dict):
    """
    Prepara il payload da scenario.
    Normalizza chiavi maiuscole/minuscole,
    converte booleani e gestisce nan come None.
    """

    def norm_val(v):
        """Converte stringhe True/False, nan → None"""
        if isinstance(v, str):
            lv = v.strip().lower()
            if lv in ("true", "1", "yes", "y"):
                return True
            if lv in ("false", "0", "no", "n"):
                return False
        if isinstance(v, float) and math.isnan(v):
            return None
        return v

    # normalizza tutte le chiavi a lowercase per uniformità
    norm_payload = {k.lower(): norm_val(v) for k, v in payload.items()}

    fixed_payload = {
        "elab_id": norm_payload.get("elab_id", ""),
        "input_folder_pre": norm_payload.get("input_folder_pre"),
        "input_folder_post": norm_payload.get("input_folder_post"),
        "output_folder": norm_payload.get("output_folder"),
        "dsm_pre_path": norm_payload.get("dsm_pre_path"),
        "dsm_post_path": norm_payload.get("dsm_post_path"),
        "buildings_path": norm_payload.get("buildings_path"),
        "output_directory": norm_payload.get("output_directory"),

        "variabili": norm_payload.get("variabili", []),

        "resampling_strategy": norm_payload.get("resampling_strategy", "best"),
        "resampling_method": norm_payload.get("resampling_method", "bilinear"),
        "merge_method": norm_payload.get("merge_method", "mean"),
        "nodata_value": norm_payload.get("nodata_value", -9999),
        "align_pixels": norm_payload.get("align_pixels", False),
        "create_hdr_files": norm_payload.get("create_hdr_files", False),
        "compression": norm_payload.get("compression", "lzw"),
        "tiled": norm_payload.get("tiled", False),
        "block_size": norm_payload.get("block_size", 256),
        "overwrite_existing": norm_payload.get("overwrite_existing", False),
        "enable_name_filtering": norm_payload.get("enable_name_filtering", False),
        "name_filter_pattern": norm_payload.get("name_filter_pattern", ""),

        # campi aggiuntivi specifici
        "damage_threshold": norm_payload.get("damage_threshold", 0.5),
        "min_overlap_percent": norm_payload.get("min_overlap_percent", 50.0),
        "height_field_name": norm_payload.get("height_field_name", "altezza"),
        "collapse_threshold_percent": norm_payload.get("collapse_threshold_percent", 50.0),
        "altezza_build": norm_payload.get("altezza_build"),
        "type_costr": norm_payload.get("type_costr"),
        "sup_field": norm_payload.get("sup_field"),
        "fid_field": norm_payload.get("fid_field"),

        # placeholders per future espansioni
        "aree_evento": [],
        "inquadramento": []
    }

    return fixed_payload

# -------------------------------------------------------------------------------- NOTEBOOK-CELL: CODE
def _create_payload():
    payload = None
    try:
        run_vars = dataiku.get_custom_variables()
        logging.debug(f"recuperato variabili custom")
        scenario_run_id = run_vars.get('scenarioTriggerRunId')
        if scenario_run_id is not None:
            logging.info(f"avvio recipe da scenario{scenario_run_id}")
            payload = json.loads(run_vars.get('scenarioTriggerParams'))
            payload = _prepare_scenario_payload(payload)
            if payload.get('elab_id') is None:
                payload["elab_id"] = f"e{scenario_run_id.replace('-', '')[:-3]}"
            else:
                payload["elab_id"] = f"e{payload['elab_id']}"
        else:
            logging.info(f"avvio recipe da flow")
            payload = _create_payload_from_config_tables()
        logging.debug(f"{json.dumps(payload)}")

        try:
            skip_keys = {'variabili', 'aree_evento', 'inquadramento'}
            for k, v in payload.items():
                if k in skip_keys:
                    continue
                # scrivi solo valori semplici
                if isinstance(v, (str, int, float, bool)) or v is None:
                    _set_project_variable(k.upper(), v)
                    # Imposta le variabili esplicite fornite in 'variabili'
            vars_list = payload.get('variabili') or []
            if isinstance(vars_list, dict):
                vars_list = [vars_list]
            for item in vars_list:
                if not isinstance(item, dict):
                    continue
                name = item.get('nome_variabile')
                val = item.get('percorso_variabile')
                if name:
                    _set_project_variable(name, val)

            logging.info(f"Variabili locali impostate da payload:")
        except Exception as ex:
            logging.warning(f"Errore impostando variabili progetto da payload: {ex}")
        return payload
    except Exception as ex:
        logging.error(f"{ex}")
        raise

# -------------------------------------------------------------------------------- NOTEBOOK-CELL: CODE
def main():
    payload = _create_payload()
    print(payload)
    if payload is None:
        logging.error("Errore nella creazione del payload")
        raise Exception("Errore nella creazione del payload")
    try:
        print("CORE ENGINE PROCESSING")
        print("Architettura Modulare")
        print("Sistema di Elaborazione Geospaziale")

        success, output_files_count, config = run_workflow(payload)

        # Ripristina stdout originale per messaggio finale
#         sys.stdout = original_stdout

        if not success:
            print(f"❌ ELABORAZIONE FALLITA - Exit Code: 1")
            sys.exit(1)

        print(f"✅ ELABORAZIONE DATASET COMPLETATA - Exit Code: 0")
        print(f"📁 Output files: {output_files_count} generati")
        print(config)
        if config:
            print(config)

            # Prepara una riga risultato serializzabile dal config
            results_row = {
                "elab_id": getattr(config, "ELAB_ID", None) or getattr(config, "elab_id", None),
                "dsm_pre_path": getattr(config, "DSM_PRE_PATH", None),
                "dsm_post_path": getattr(config, "DSM_POST_PATH", None),
                "output_folder": getattr(config, "OUTPUT_FOLDER", None),
                "merge_subfolder": getattr(config, "MERGE_SUBFOLDER", None),
                "resampling_strategy": getattr(config, "RESAMPLING_STRATEGY", None),
                "resampling_method": getattr(config, "RESAMPLING_METHOD", None),
                "merge_method": getattr(config, "MERGE_METHOD", None),
                "nodata_value": getattr(config, "NODATA_VALUE", None),
                "align_pixels": getattr(config, "ALIGN_PIXELS", None),
                "create_hdr_files": getattr(config, "CREATE_HDR_FILES", None),
                "compression": getattr(config, "COMPRESSION", None),
                "tiled": getattr(config, "TILED", None),
                "block_size": getattr(config, "BLOCK_SIZE", None),
                "overwrite_existing": getattr(config, "OVERWRITE_EXISTING", None),
            }
            try:
                df_results = pd.DataFrame([results_row])
                db_results = dataiku.Dataset("db_results")
                db_results.write_with_schema(df_results)
                print("✅ db_results aggiornato")
            except Exception as e:
                print(f"⚠️ Impossibile scrivere db_results: {e}")
        sys.exit(0)

    except Exception as e:
        # Ripristina stdout in caso di errore critico
#         sys.stdout = original_stdout
        print(f"❌ ERRORE CRITICO - Exit Code: 1")
        print(f"Errore: {str(e)}")
        sys.exit(1)

# -------------------------------------------------------------------------------- NOTEBOOK-CELL: CODE
# AVVIA MAIN
main()

# -------------------------------------------------------------------------------- NOTEBOOK-CELL: CODE
# Recipe outputs
# vxkciquq = dataiku.Folder("vxKCiqUq")
# vxkciquq_info = vxkciquq.get_info()